# ZTF Sparse photometry for Asteroids

This notebook uses ZTF Forced Photometry Service, whose manual can be found here:
<https://irsa.ipac.caltech.edu/data/ZTF/docs/ztf_forced_photometry.pdf>

### Useful packages

In [ ]:
import requests
import pandas as pd
from jdcal import MJD_0, jd2gcal
from IPython.display import clear_output
from os.path import exists
import json

## Asteroid Info and Datasets

In [ ]:
#Write only the id of asteroid:
asteroid_id=2616

#Import the ZTF file corresponds to the above asteroid:
ZTF_file='MOST/'+str(asteroid_id)+'_ZTF.tbl'

#Import Time Range
#For ZTF: Time Range = 2017-10-15 to present
start_date="2017-10-15"
end_date="2025-04-11"

### Download and read file from MOST

The search of ZTF images, that contain the selected asteroid, was performed using the Moving Object Search Tool (MOST). This tool is available at: <https://irsa.ipac.caltech.edu/applications/MOST/>
The downloaded file should have name; <asteroid_id>_ZTF.tbl and be placed in; MOST folder.

In [ ]:
if exists(ZTF_file):
    print("Output file for this asteroid from the MOST already exists!")
else:
    MOST_r = requests.get("https://irsa.ipac.caltech.edu/cgi-bin/MOST/nph-most?catalog=ztf&input_type=naifid_input&obj_naifid="+str(20000000+asteroid_id)+"&obs_begin="+start_date+"&obs_end="+end_date+"&output_mode=Brief")
    MOST_a=MOST_r.text
    MOST_url=MOST_a.partition('\n')[0].split('=', 1)[1].strip('\"').replace(' "','')
    print("The MOST request is successful!")

In [ ]:
if exists(ZTF_file):
    print("Output file for this asteroid from the MOST already exists!")
else:
    MOST_s=requests.get(MOST_url+"/results.tbl")
    with open(ZTF_file, 'w') as file:
        file.write(MOST_s.text)
    print("The MOST output file was saved!")

In [ ]:
if exists(ZTF_file):
    data=pd.read_table(ZTF_file, header=None, skiprows=[14,15,16], comment='\\',  delim_whitespace=True, names=['Image_ID', 'date_obs', 'time_obs', 'mjd_obs', 'ra_obj', 'dec_obj', 'sun_dist', 'geo_dist', 'dist_ctr', 'phase', 'vmag', 'image_url', 'postcard_url', 'region_file'])
    dataR=data[data['Image_ID'].str.contains("_zr_")]
else:
    print("No MOST output file! \n You must follow the above instructions.")
dataR

### ZTF Forced Photometry Service Request
The following command will request a forced photometry for the asteroid with individual submissions. Each individual submission consists of supplying a single R.A., Dec. position, a time-range of interest, and your credentials. You can submit requests using also the graphical user interface (GUI) form:
<https://ztfweb.ipac.caltech.edu/cgi-bin/requestForcedPhotometry.cgi>
The username, password for initiating the GUI service is: **ztffps**, **dontgocrazy!**

These credentials are fixed and should not be confused with your personal password, which you should have received by registering. The requestor’s email address and password you supply on the form are precisely those you registered with.

**It takes some time if the dataset is large! It will not duplicate the job if it has been done before.**

In [ ]:
#Insert here your personal credentials
email='user@email.com'
userpass='yourpassword'

In [ ]:
#Run it only ones!
if exists('ZTF/ForcedPhot/'+str(asteroid_id)+'_ZTF_FP.txt'):
    print("A file with Forced Photometry data exists!")
else:
    for i in dataR.index:
        clear_output(wait=True)
        print("Progress "+str(i)+"/"+str(max(dataR.index)))

        requests.get('https://ztfweb.ipac.caltech.edu/cgi-bin/requestForcedPhotometry.cgi', auth=('ztffps', 'dontgocrazy!'), params={'ra': str(dataR['ra_obj'][i]), 'dec': str(dataR['dec_obj'][i]), 'jdstart': str(dataR['mjd_obs'][i]+MJD_0), 'jdend': str(dataR['mjd_obs'][i]+MJD_0), 'email': email, 'userpass': userpass})

    clear_output(wait=True)
    print("Done!")

### Now, you have to wait...

### Check the status or history of your submissions
Now you have to check your email. You will receive an email from ZTF Forced Photometry service for each submission.

You can check the status or history of your submissions using the utility:
<https://ztfweb.ipac.caltech.edu/cgi-bin/getForcedPhotometryRequests.cgi>
The username, password for initiating this service is also: **ztffps**, **dontgocrazy!**

Due to possible excessive database loads, the job queue is refreshed every hour. It is refreshed on the hour according to wall clock. Therefore, the job status you see could be up to one hour old. Furthermore, only requests submitted within the last month can be accessed.

### Save and read the result at once
**IMPORTANT: Go to the history of your submission copy the cells of your last submissions and paste them to a file with name** <asteroid_id>_ZTF_FP.txt in; ForcedPhot folder.
**Then you can run ZTF_photometry.inyb**


**Note:** Be aware that you have to wait 1 hour in order to see your full status of your history!

Then the following commands will read the file and retrieve the data.

**Exit and warning codes (the "procstatus" column)**
Overall exit and warning codes from execution of your request are provided in the log file that accompanies the output table. Processing status codes are also provided on a per-epoch basis. This is the procstatus column in the output table. These codes are defined below.
- 0 => Successful execution
- 56 => One or more epochs have photometry measurements that may be impacted by bad (including NaN'd) pixels
- 57 => One or more epochs had no reference image catalog source falling with 5 arcsec
- 58 => One or more epochs had a reference image PSF-catalog that does not exist in the archive
- 59 => One or more epochs may have suspect photometric uncertainties due to early creation date of difference image in production
- 60 => One or more epochs had upsampled diff-image PSF dimensions that were not odd integers
- 61 => One or more epochs had diff-image cutouts that were off the image or too close to an edge
- 62 => Requested start JD was before official survey start date [3/17/18] and was reset to 2018-03-17T00:00:00.0 UT
- 63 => No records (epochs) returned by database query
- 64 => Catastrophic error (see log output)
- 65 => Requested end JD is before official survey start date [3/17/18]
- 255 => Database connection or query execution error (see log output)